In [4]:
import os
import numpy as np
import pandas as pd
from pathlib import Path
from bids import BIDSLayout
from nilearn.interfaces.fmriprep import load_confounds
from nilearn.masking import intersect_masks
from nilearn.image import load_img
from nilearn.glm.first_level import FirstLevelModel
import nibabel as nib

import pandas as pd
from nilearn.glm.second_level import SecondLevelModel
from nilearn.glm.second_level import make_second_level_design_matrix  # convenience
from nilearn import plotting

from nilearn.plotting import plot_design_matrix, plot_design_matrix_correlation
import matplotlib
matplotlib.use("Agg")  # headless-safe on clusters


bids_root = Path("/orange/ruogu.fang/leem.s/EmotionVideo/Kamitani/")  # dataset root (contains sub-XX/, sub-XX/ses-YY/, etc.)
deriv = os.path.join(bids_root, "derivatives")

# Allow derivatives so layout can see fMRIPrep outputs
layout = BIDSLayout(bids_root, validate=False, derivatives=True)

In [5]:
def collect_subject(subject):
    # All preproc bolds in standard space (adjust entities to match your outputs)
    imgs = layout.get(subject=subject, datatype="func", suffix="bold",space="MNI152NLin2009cAsym", desc="preproc",
                      extension=[".nii", ".nii.gz"], return_type="file")

    # One mask per run
    masks = layout.get(subject=subject, datatype="func", suffix="mask",space="MNI152NLin2009cAsym", desc="brain",
                       extension=[".nii", ".nii.gz"], return_type="file")

    # Confounds tsv per run
    confs = layout.get(subject=subject, suffix="timeseries",
                       desc="confounds", extension=".tsv",
                       return_type="file")

    # Events per run (from the raw BIDS side, not derivatives)
    events = layout.get(subject=subject, suffix="events2",
                        extension=".tsv", return_type="file")

    # TR from the first BOLD’s JSON
    meta = layout.get_metadata(imgs[0])
    tr = float(meta.get("RepetitionTime"))
    
    print("## Data Import Complete ##", flush=True)
    return imgs, masks, confs, events, tr


def get_confounds(img_files):
    # Strategy "simple" is a common starting point; "scrubbing" adds FD/DVARS censoring
    conf_list, sample_masks = load_confounds(
        img_files,
        strategy=["motion", "high_pass", "wm_csf"],  # add "global_signal" if you intend GSR
        motion='basic', 
        scrub=True, fd_threshold=0.5, std_dvars_threshold=1.5,
        demean=True
    )
    return conf_list, sample_masks

def subject_mask(mask_files):
    return intersect_masks(mask_files, threshold=1.0, connected=False)  # strict intersection


def load_events(event_files):
    return [pd.read_csv(ef, sep="\t") for ef in event_files]


def fit_first_level(imgs, events, confounds, sample_masks, mask_img, tr):
    
    fm = FirstLevelModel(
        t_r=tr, mask_img=mask_img,
        hrf_model="spm", drift_model="cosine", high_pass=0.008,
        smoothing_fwhm=None, noise_model="ar1",
        minimize_memory=True, n_jobs=2, verbose=1,
        signal_scaling=False
    )

    fm.fit(imgs, events=events, confounds=confounds)

    return fm

def save_contrast(fm, contrast, out_dir, label):
    out_dir = Path(out_dir); out_dir.mkdir(parents=True, exist_ok=True)
    # Ask for both effect and variance; this is the fixed-effects result across runs
    out = fm.compute_contrast(contrast, output_type="all")

    eff = out["effect_size"]
    var = out["effect_variance"]
    z   = out["z_score"]  # handy for quick QA

    eff_p = out_dir / f"{label}_effect.nii.gz"
    var_p = out_dir / f"{label}_variance.nii.gz"
    z_p   = out_dir / f"{label}_zmap.nii.gz"

    nib.save(eff, eff_p); nib.save(var, var_p); nib.save(z, z_p)
    return eff_p, var_p, z_p

In [10]:
subjects = ["01", "02", "03", "04", "05"]
contrasts = {
    
    "pleasant>neutral": "pleasant - neutral",
    "pleasant>rest": "pleasant - rest",
    "pleasant>unpleasant": "pleasant - unpleasant",
    

    "rest>neutral": "rest - neutral",
    "rest>unpleasant": "rest - unpleasant",
    "rest>pleasant": "rest - pleasant",
    
    "neutral>unpleasant": "neutral - unpleasant",
    "neutral>pleasant": "neutral - pleasant",
    "neutral>rest": "neutral - rest",
    
    "unpleasant>pleasant": "unpleasant - pleasant",
    "unpleasant>rest": "unpleasant - rest",
    "unpleasant>neutral": "unpleasant - neutral"}

eff_map_per_sub = {}
effect_paths = {name: {} for name in contrasts}

for sub in subjects:
    '''
    imgs, masks, confs, events, tr = collect_subject(sub)
    conf_list, sample_masks = get_confounds(imgs)
    mask_img = subject_mask(masks)
    events_list = load_events(events)
    
    print("## Fitting the first level analysis for subject-{}".format(sub))
    fm = fit_first_level(imgs, events_list, conf_list, sample_masks, mask_img, tr)
    
    report = fm.generate_report(
    contrasts=contrasts,              # one or many
    title="sub-{} first-level GLM".format(sub),   # anything you like
    bg_img="MNI152TEMPLATE",          # or a NIfTI path
    alpha=0.05,                 # overall FWER level
    height_control="bonferroni",# voxelwise Bonferroni correction
    cluster_threshold=None,       # optional: remove clusters < 15 vox
    two_sided=True,
    plot_type="glass",                # "slice" or "glass"
    report_dims=(1600, 800),
)
    '''
    
    outroot = "/orange/ruogu.fang/leem.s/EmotionVideo/Kamitani/derivatives/nilearn/no_smooth"
    sub_out = os.path.join(outroot,f"sub-{sub}")
    outdir = Path(sub_out)
    outdir.mkdir(parents=True, exist_ok=True)
    '''
    
    # In notebooks, just display(report)
    # To save a standalone HTML file:
    report.save_as_html(os.path.join(sub_out, "firstlevel_report.html"))
    # Or open directly in a browser:
    # report.open_in_browser()
    
    dm_dir = os.path.join(sub_out, "design_matrices")
    os.makedirs(dm_dir, exist_ok=True)

    # fm.design_matrices_ is a list: one pandas DataFrame per run
    for r_ix, dm in enumerate(fm.design_matrices_):
        dm_png   = os.path.join(dm_dir, f"design_run-{r_ix+1:02d}.png")
        corr_png = os.path.join(dm_dir, f"design_corr_run-{r_ix+1:02d}.png")

        # Design matrix image
        plot_design_matrix(dm, output_file=dm_png)  # writes and closes the figure

        # Correlation heatmap between regressors (drifts/constant omitted)
        plot_design_matrix_correlation(dm, output_file=corr_png)  # writes and closes
    
    '''
    for label, expr in contrasts.items():
        # Ask for both effect and variance if you like; effect is sufficient for SecondLevelModel
        '''
        out = fm.compute_contrast(expr, output_type="all")  # effect_size, variance, z_score
        eff = out["effect_size"]     # fixed-effects effect across runs
        z   = out["z_score"]         # optional QA
        
        '''
        eff_p = os.path.join(sub_out, f"{label}_effect.nii.gz")
        z_p   = os.path.join(sub_out, f"{label}_zmap.nii.gz")
        '''
        nib.save(eff, eff_p)
        nib.save(z, z_p)
        '''
        effect_paths[label][sub] = str(eff_p) # formatted like  {'rest>neutral': {'01': '/orange/ruogu.fang/leem.s/EmotionVideo/Kamitani/derivatives/nilearn/sub-01/rest>neutral_effect.nii.gz'}}
        

In [11]:
from scipy.stats import norm
from nilearn.reporting import get_clusters_table
from nilearn.glm import threshold_stats_img
print("## Perform 2-nd Level Analysis ##")
        
group_out = Path(os.path.join("/orange/ruogu.fang/leem.s/EmotionVideo/Kamitani/derivatives/nilearn/no_smooth","group"))
group_out.mkdir(parents=True, exist_ok=True)

   
for label in contrasts.keys():
    # Build the second-level input table
    rows = []
    for sub in subjects:
        rows.append({
            "subject_label": "sub-{}".format(sub),
            "map_name": label,
            "effects_map_path": effect_paths[label][sub],
        })
    second_df = pd.DataFrame(rows)

    # Design matrix (intercept-only -> one-sample t-test)
    design = make_second_level_design_matrix(second_df["subject_label"])
    # Alternatively: design = pd.DataFrame({"intercept": [1] * len(second_df)})

    # Fit and test
    #slm = SecondLevelModel(smoothing_fwhm=8.0, n_jobs=4)
    slm = SecondLevelModel(smoothing_fwhm=6.0, n_jobs=4)
    slm.fit(second_level_input=second_df, design_matrix=design)

    # One-sample test on this contrast (use the intercept)
    z_map = slm.compute_contrast(second_level_contrast="intercept",
                                 first_level_contrast=label,
                                 output_type="z_score")  # or "stat"/"p_value"
    
    z_path = group_out / f"group_{label}_zmap.nii.gz"
    nib.save(z_map, z_path)
    
    _, zthr_bonf = threshold_stats_img(
    z_map,
    alpha=0.05,
    height_control="bonferroni",
    two_sided=False,   # keep consistent with your report
    )


    # (Optional) quick plot
    p_val = 0.001
    p001_unc = norm.isf(p_val)
    fig = plotting.plot_glass_brain(z_map, threshold=zthr_bonf, colorbar=True, display_mode="lyrz",
                                    title=f"Group {label} (Bonferroni, z>{zthr_bonf:.2f}")
    fig.savefig(str(group_out / f"group_{label}_glass.png"))

    # ---------- NEW: generate an HTML report for this contrast ----------
    report = slm.generate_report(
        contrasts={"group_mean": "intercept"},  # 2nd-level contrast
        first_level_contrast=label,             # name used in your subject maps
        title=f"Group: {label}",
        bg_img="MNI152TEMPLATE",
        threshold=1.5, 
        alpha=0.05,                 # overall FWER level
        height_control="fpr",
        display_mode='y',
        cut_coords=[-6, -4, -2, 0, 2, 40, 60],
        two_sided=False,
        plot_type="slice", report_dims=(1600, 800),
    )
    report.save_as_html(str(group_out / f"group_{label}_report.html"))

    # (Optional) add a CSV of cluster peaks for the z-map
    tbl = get_clusters_table(z_map, stat_threshold=zthr_bonf, two_sided=False)
    tbl.to_csv(group_out / f"group_{label}_clusters.csv", index=False)
    


## Perform 2-nd Level Analysis ##


/scratch/local/15082638/ipykernel_1766144/88160834.py:38: UserWarning: The given float value must not exceed 4.999143745996977. But, you have given threshold=5.062426559825845.
  _, zthr_bonf = threshold_stats_img(
/scratch/local/15082638/ipykernel_1766144/88160834.py:49: UserWarning: empty mask
  fig = plotting.plot_glass_brain(z_map, threshold=zthr_bonf, colorbar=True, display_mode="lyrz",
/scratch/local/15082638/ipykernel_1766144/88160834.py:54: UserWarning: 'threshold=1.5' will not be used with 'height_control='fpr''. 'threshold' is only used when 'height_control=None'. Set 'threshold' to '3.09' to avoid this warning.
  report = slm.generate_report(
/scratch/local/15082638/ipykernel_1766144/88160834.py:54: UserWarning: 'threshold=1.5' will not be used with 'height_control='fpr''. 'threshold' is only used when 'height_control=None'. Set 'threshold' to '3.0' to avoid this warning.
  report = slm.generate_report(
/scratch/local/15082638/ipykernel_1766144/88160834.py:70: UserWarning: T

In [5]:
from nilearn.glm.second_level import non_parametric_inference
from nilearn.image import math_img
from scipy.stats import norm

group_out = Path(os.path.join("/orange/ruogu.fang/leem.s/EmotionVideo/Kamitani/derivatives/nilearn/no_smooth","group"))
group_out.mkdir(parents=True, exist_ok=True)
vmax = 2.69  # ~= -np.log10(1 / 500)

for label in contrasts.keys():
    # Build the second-level input table
    rows = []
    for sub in subjects:
        rows.append({
            "subject_label": "sub-{}".format(sub),
            "map_name": label,
            "effects_map_path": effect_paths[label][sub],
        })
    second_df = pd.DataFrame(rows)

    # Design matrix (intercept-only -> one-sample t-test)
    design = make_second_level_design_matrix(second_df["subject_label"])
    # Alternatively: design = pd.DataFrame({"intercept": [1] * len(second_df)})


    # ----- Nonparametric voxel-level FWE (max-T), no cluster-forming threshold -----
    # Returns a NIfTI of -log10(p_FWE) for voxels.
    neglog10_p_vox = non_parametric_inference(
        second_level_input=second_df,
        design_matrix=design,
        second_level_contrast="intercept",
        first_level_contrast=label,
        model_intercept=True,
        n_perm=10000,              # adjust (>=5000 common); higher = slower but stabler
        two_sided_test=False,     # set True if you want two-sided
        n_jobs=8,
        smoothing_fwhm=10.0,
        threshold=None,           # <-- voxel-level FWE (max-T)
        tfce=False
    )
    # Save voxel-level FWE results
    neglog10_p_vox_path = group_out / f"group_{label}_neglog10p_voxFWE.nii.gz"
    nib.save(neglog10_p_vox, neglog10_p_vox_path)

    # Optional: convert -log10(p) -> p for downstream uses
    p_img_vox = math_img("10 ** (-img)", img=neglog10_p_vox)
    nib.save(p_img_vox, group_out / f"group_{label}_p_voxFWE.nii.gz")

    # Quick plot at p_FWE < .05 => -log10(p) > 1.3010
    fig = plotting.plot_glass_brain(
        neglog10_p_vox, threshold=1.30103, colorbar=True, display_mode="lyrz",
        title=f"Permutation voxel-FWE {label} (-log10 p)"
    )
    fig.savefig(str(group_out / f"group_{label}_perm_voxFWE_glass.png"))

    # ----- Nonparametric cluster-level FWE (max-size / max-mass) -----
    # Choose a cluster-defining voxelwise threshold. Using z ~ 3.09 (p<.001 one-sided) here:
    p001_unc = 0.001
    z_cdt = norm.isf(p001_unc)  # ~3.09 for one-sided
    perm_outputs = non_parametric_inference(
        second_level_input=second_df,
        design_matrix=design,
        second_level_contrast="intercept",
        first_level_contrast=label,
        model_intercept=True,
        n_perm=10000,
        two_sided_test=False,
        n_jobs=8,
        smoothing_fwhm=10.0,
        threshold=p001_unc,          # <-- CDT enables cluster-level inference
        tfce=False
    )
    # perm_outputs is a dict-like with keys e.g. 'logp_max_size' (cluster-extent FWE),
    # 'mass' & 'logp_max_mass' (cluster-mass FWE). Save the most used one:
    logp_max_size = perm_outputs["logp_max_size"]
    nib.save(logp_max_size, group_out / f"group_{label}_neglog10p_clustSizeFWE.nii.gz")

    # Visualize cluster-extent FWE at p_FWE < .05
    fig = plotting.plot_glass_brain(
        logp_max_size, threshold=1.30103, colorbar=True, display_mode="lyrz",
        title=f"Permutation cluster-FWE (extent) {label} (-log10 p)"
    )
    fig.savefig(str(group_out / f"group_{label}_perm_clustSizeFWE_glass.png"))

    # (Optional) use cluster MASS (often more sensitive than pure extent)
    logp_max_mass = perm_outputs["logp_max_mass"]
    nib.save(logp_max_mass, group_out / f"group_{label}_neglog10p_clustMassFWE.nii.gz")

    # ----- Nonparametric TFCE (no CDT needed) -----
    perm_tfce = non_parametric_inference(
        second_level_input=second_df,
        design_matrix=design,
        second_level_contrast="intercept",
        first_level_contrast=label,
        model_intercept=True,
        n_perm=10000,
        two_sided_test=False,
        n_jobs=8,
        smoothing_fwhm=10.0,
        threshold=None,           # ignored when tfce=True
        tfce=True                 # <-- TFCE mode
    )
    # Returns at least 'tfce' and 'logp_max_tfce'
    tfce_score = perm_tfce["tfce"]
    logp_max_tfce = perm_tfce["logp_max_tfce"]
    nib.save(tfce_score,   group_out / f"group_{label}_tfce_score.nii.gz")
    nib.save(logp_max_tfce, group_out / f"group_{label}_neglog10p_TFCEFWE.nii.gz")

    fig = plotting.plot_glass_brain(
        logp_max_tfce, vmax=vmax, threshold=1.30103, colorbar=True, display_mode="lyrz",
        title=f"Permutation TFCE-FWE {label} (-log10 p)"
    )
    fig.savefig(str(group_out / f"group_{label}_perm_TFCEFWE_glass.png"))


/scratch/local/13843812/ipykernel_2601491/960512911.py:49: UserWarning: empty mask
  fig = plotting.plot_glass_brain(
/scratch/local/13843812/ipykernel_2601491/960512911.py:59: UserWarning: Data array used to create a new image contains 64-bit ints. This is likely due to creating the array with numpy and passing `int` as the `dtype`. Many tools such as FSL and SPM cannot deal with int64 in Nifti images, so for compatibility the data has been converted to int32.
  perm_outputs = non_parametric_inference(
/red/ruogu.fang/leem.s/conda/envs/fmri/lib/python3.13/site-packages/joblib/memory.py:326: UserWarning: Data array used to create a new image contains 64-bit ints. This is likely due to creating the array with numpy and passing `int` as the `dtype`. Many tools such as FSL and SPM cannot deal with int64 in Nifti images, so for compatibility the data has been converted to int32.
  return self.func(*args, **kwargs)
/scratch/local/13843812/ipykernel_2601491/960512911.py:78: UserWarning: empt

KeyboardInterrupt: 